<a href="https://colab.research.google.com/github/francji1/01ZLMA/blob/main/R/01ZLMA_ex06_ANCOVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

01ZLMA - Exercise 06

Exercise 06 of the course 01ZLMA. 

# Analysis of Covariance (ANCOVA)

Assume dataset from A. J. Dobson - An Introduction to GLM, Exercise 5, that contains observations of point evaluation based on three different methods $A$, $B$, and $C$.

In [ ]:
library(tidyverse)
library(Matrix)
library(MASS)

In [ ]:
data_trening <- "https://raw.githubusercontent.com/francji1/01ZLMA/main/data/data_trenink.csv"
data    <- read.table(data_trening, header = T, sep = ";")
summary(data)
head(data)


In [ ]:
data <- data %>% mutate(group = factor(group))
str(data)
attach(data)


In [ ]:
ggplot(data, aes(x=group, y=y, fill = group)) + 
  geom_boxplot()+
  labs(title="Achievement score boxplot",x="Method", y = "Achievement score")+
  geom_jitter(shape=16, position=position_jitter(0.2)) +
  stat_summary(fun=mean, geom="point", shape=23, size=3) +
  theme_classic()

In [ ]:
ggplot(data, aes(x=x, y=y, color = group, shape = group)) + 
  geom_point()+
  labs(title="Achievement score scatterplot",x="Aptitude scores", y = "Achievement score")+
  theme_classic()

Our task is to find out whether the individual methods differ from each other. 
At first,  we will run the analysis by the help of  derived formulas from the lecture, then we will use the built in functions from `R`.

## ANCOVA - 01ZLMA Lecture 07

###  Saturated model
Task: Estimate regression coefficients and compute deviance statistcs.

In [ ]:
one <- rep(1,7)
zero <- rep(0,7)

Z <- matrix(c(one,zero,zero,zero,one,zero,zero,zero,one),ncol=3)
G <- diag(c(1/7,1/7,1/7))
P <- diag(rep(1,21)) - Z %*% G %*% t(Z)
A <- x %*% P %*% x
A1 <- solve(A)

beta <- as.numeric(A1 %*% x %*% P %*% y)
beta
u1 <- G %*% t(Z) %*% y
u2 <- G %*% t(Z) %*% x %*% beta
u <- u1 - u2
u

y.hat <- beta*x + Z%*%u
D <- crossprod(y-y.hat)
D


###  Zúžený model 


Odhadneme parametry a spočteme deviační statistiku modelu za platnosti $H_0$:

In [ ]:
Z0 <- rep(1,21)
G0 <- 1/21
P0 <- diag(rep(1,21)) - 1/21 * matrix(1,21,21)
A <- x %*% P0 %*% x
A1 <- solve(A)

beta0 <- as.numeric(A1 %*% x %*% P0 %*% y)
beta0

u0 <- mean(y) - mean(x)*beta0
u0

y.hat0 <- beta0*x + Z0*u0
D0 <- crossprod(y-y.hat0)
D0


### Porovnání modelů pomocí F statistiky

In [ ]:
F <- (D0-D)/2/(D/(21-3-1))
F
CV <- qf(0.95,2,17)
CV
p.val <- 1 - pf(F,2,17)
p.val

Hypotézu, že mezi metodami není rozdíl zamítáme.

### Test významnosti vysvětlujíí proměnné $x$ 


Otestujme hypotézu $H_0: \beta = 0$ pomocí porovnání obecného modelu s modelem za platnosti $H_0$

In [ ]:
mod.x <- lm(y~group-1)
summary(mod.x)

jehož deviace je

In [ ]:
Dx <- crossprod(y-fitted(mod.x))
Dx

Pro $F$-statistiku porovnávající oba modely platí

In [ ]:
F <- (Dx-D)/1/(D/(21-3-1)); F
CV <- qf(0.95,1,17); CV
p.val <- 1 - pf(F,1,17); p.val

a hypotézu tedy zamítáme, tzn. proměnná $x$ je v modelu významná. Srovnání provedeme pro ilustraci ještě pomocí funkce `anova`

In [ ]:
modAOC <- lm(y~x+group-1)  # obecný model
anova(mod.x, modAOC, test = "F")

### Vícenásobné porovnávání (Bonferroni) 

Vraťme se k obecnému modelu, a protože byla zamítnuta hypotéza rovnosti efektů jednotlivých metod, proveďme vícenásobné porovnávání s cílem zjistit, které dvojice se významně liší.

Hodnoty pevných efektů pro jednotlivé metody jsou

In [ ]:
u.1 <- u[1]; u.2 <- u[2]; u.3 <- u[3]

a tabulka jejich rozdílů je

In [ ]:
difu1<-c(u.1-u.2,u.1-u.3); difu1<-abs(difu1)
difu2<-c(0,u.2-u.3); difu2<-abs(difu2)
meanabs<-rbind(difu1,difu2)
c.names<-c("mean.g.B","mean.g.C")
r.names<-c("mean.g.A","mean.g.B")
dimnames(meanabs)<-list(r.names,c.names)
meanabs

Spočteme kritické hodnoty pro Bonferoniho metodu vícenásobného porovnávání

In [ ]:
sigma.hat <- D/(21-3-1); sigma.hat
t.val <- qt(1-0.05/6,17); t.val

x.m1 <- mean(x[group=="A"])
x.m2 <- mean(x[group=="B"])
x.m3 <- mean(x[group=="C"])

n<-tapply(y, group, length)   # počet pozorování v jednotlivých kategoriích "group"

x.m <- c(rep(x.m1,n[1]),rep(x.m2,n[2]),rep(x.m3,n[3]))

Exx <- crossprod(x-x.m); Exx

BF.12 <- sqrt(sigma.hat)*t.val*sqrt(1/n[1] + 1/n[2] + 1/Exx*(x.m1-x.m2)^2)
BF.13 <- sqrt(sigma.hat)*t.val*sqrt(1/n[1] + 1/n[3] + 1/Exx*(x.m1-x.m3)^2)
BF.23 <- sqrt(sigma.hat)*t.val*sqrt(1/n[2] + 1/n[3] + 1/Exx*(x.m2-x.m3)^2)

#tabulka hodnot BF
BF1<-c(BF.12,BF.13)
BF2<-c(0,BF.23)
BF<-rbind(BF1,BF2)
dimnames(BF)<-list(r.names,c.names)
BF

Porovnáním hodnot v obou tabulkách zjistíme, které dvojice se významně liší

In [ ]:
SIGNIF<-meanabs>BF; SIGNIF


Významně se liší metoda $A$ od $B$ a metoda  $A$ od $C$.

Obrázek dat proložených modelem

In [ ]:
plot(x, y, pch = c(15:17)[group], col = c("red","blue","black")[group], 
     xlab = "Před tréninkem", ylab = "Po tréninku")
legend("topleft",inset = .01, bty="n", legend = c("metoda A", "metoda B", "metoda C"), 
       pch = c(15:17), col = c("red","blue","black"), cex=0.9)
abline(coef = c(u[1],beta),col = "red")
abline(coef = c(u[2],beta),col = "blue")
abline(coef = c(u[3],beta),col = "black")

## ANOVA - pomocí funkcí `R` 
<!-- ######################## -->

Pro ilustraci proveďme i analýzu rozptylu ANOVA, tzn. nebudeme uvažovat proměnnou x.

In [ ]:
is.factor(group)  #ověření, že se jedná o faktorovou proměnnou

In [ ]:
group = as.factor(group)

Dvě možnosti, jak získat tabulku analýzy rozptylu jsou 

In [ ]:
aov_m1 <- aov(y~group)
summary(aov_m1)

In [ ]:
lm_m1 <- lm(y~group)
anova(lm_m1)

In [ ]:
opar <- par(mfrow=c(2,2))
plot(aov(y~group))
#plot(lm_m1)
par(opar)

Závěr: proměnná group je významná, neboli jednotlivé metody tréninku se mezi sebou liší i bez započtení efektu proměnné $x$.

Spočteme průměry pro jednotlivé skupiny tréninku 

In [ ]:
model.tables(aov(y~group), type="means")

nebo přímo

In [ ]:
tapply(y,group,mean)

a provedeme vícenásobné porovnávání. Tentokráte Tukeyovo HSD.

In [ ]:
Tukey_CI <- TukeyHSD(aov_m1, c("group"), ordered = FALSE, conf.level = 0.95)
Tukey_CI


Výstupem jsou 95% intervaly spolehlivosti pro rozdíl průměrů a p-hodnoty testu hypotéz, že je daný rozdíl nulový. Opět vidíme rozdílnost mezi skupinami $A$, $B$ a $A$,$C$. Na obrázku to vypadá následovně

In [ ]:
plot(Tukey_CI)

Pokud zobrazený interval spolehlivosti neobsahuje 0, příslušný rozdíl je statisticky významný.

Další možnost je použít balík `multcomp`, který bude fungovat i pro model ANCOVA.

In [ ]:
install.packages("multcomp")
library(multcomp)

In [ ]:
amod<-aov(y~group) # vytvoříme model
# vícenásobné porovnávání Tukey
Tukey <- glht(amod, linfct = mcp(group = "Tukey"))
summary(Tukey)

Případně zobrazíme krabicové diagramy pro vícenásobné porovnávání

In [ ]:
install.packages("multcompView")
library(multcompView)
multcompBoxplot(y~group, data=data,compFn="TukeyHSD",sortFn="mean", decreasing=TRUE)

Opět vidíme, že metoda $A$ se liší od $B$ a $C$.

Pro ilustraci ještě porovnejme Bonferroniho metodu a klasický dvouvýběrový t-test

In [ ]:
pairwise.t.test(y, group, p.adjust.method="bonferroni")
pairwise.t.test(y, group, "none")


Vidíme poměrně významný rozdíl v p-hodnotách.

### Fisher LSD

In [ ]:
#install.packages("agricolae")
library(agricolae)
#LSD_out <- LSD.test(aov_m1,"group", p.adj="bonferroni")
#LSD_out
LSD_out <- LSD.test(aov_m1,"group",18,1.5)
LSD_out



## ANCOVA - pomocí funkcí `R` 


In [ ]:
modAOC <- lm(y~x+group-1)
summary(modAOC)
anova(modAOC)

Závěr: obě proměnné jsou významné.

Odhadnuté koeficienty

In [ ]:
coef<-summary(modAOC)$coefficients
 coef

Model za platnosti $H_0$

In [ ]:
modAOC.0 <- lm(y~x)
summary(modAOC.0)
anova(modAOC.0)

Porovnání modelů:

In [ ]:
anova(modAOC.0, modAOC, test = "F")

Porovnání lze provést i přímo

In [ ]:
modAOC1 <- lm(y~x+group)
#summary(modAOC1)
anova(modAOC1)

Vícenásobné porovnávání (Tukey HSD)

In [ ]:
amod<-aov(y~x+group)
posthoc <- glht(amod, linfct = mcp(group = "Tukey"))
summary(posthoc)
confint(posthoc)

In [ ]:
plot(posthoc)

Pozor, následující funkce funguje pouze pro model ANOVA!

In [ ]:
CI<-TukeyHSD(aov(y~x+group), which="group")
CI

Je třeba také provést analýzu reziduí a influenčních pozorování. Model ANCOVA předpokládá normalitu reziduí a také společnou hodnotu parameru $\sigma^2$.

In [ ]:
X<-model.matrix(modAOC)
n<-nrow(X); p<-ncol(X)
fit <- predict(modAOC, type = "response")

In [ ]:
# pákové body
hii <- hatvalues(modAOC)
# Kritérium pro páková pozorování
Infl<-hii>2*p/n; Infl
# Cookova vzdalenost
c.d <- cooks.distance(modAOC)


Grafické zobrazení

In [ ]:
par(mfrow=c(1,2))

plot(hii,col="red", cex=1.5, lwd=2, ylim = c(0,0.4)) 
abline(2*p/(n),0)

plot(c.d,col="red", cex=1.5, lwd=2, ylim = c(0,0.7))
abline(8/(n-2*p),0)

neukazuje žádné podezřelé body.

Ještě spočteme studentizovaná rezidua 

In [ ]:
res <- rstudent(modAOC)

a otestujeme jejich normalitu pomocí Shapirova testu

In [ ]:
shapiro.test(res)

Hypotéza normality reziduí nebyla zamítnuta. Provedeme ještě grafickou analýzu reziduí.

In [ ]:
par(mfrow=c(2,2))
# QQplot
qqnorm(res)
qqline(res)
# rezidua vs. fitted values
plot(fitted(modAOC),res, col="red", xlab="Predikované hodnoty", ylab="Rezidua", cex=1.5, lwd=2)
abline(0,0)
# rezisua vs. x
plot(x,res,col="red", xlab="Proměnná x", ylab="Rezidua", cex=1.5, lwd=2)
abline(0,0)
# rezisua vs. group
plot(group,res,col="red", xlab="Metoda tréninku", ylab="Residuals", cex=1.5, lwd=2)
abline(0,0)

Ani zde není žádný očividný probklém. Předpoklady modelu tedy můžeme považovat za splněné.

Další možnost, jka zobrazit rezidua je např.

In [ ]:
plot(modAOC, which = 1)